In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install transformers tqdm

In [ ]:
# !unzip -q -o "/content/drive/My Drive/ML_team_project_final/guardian_top100_scraping.zip" -d "/content/"

Cell 1: 라이브러리 임포트 및 기본 설정

In [ ]:
# [Cell 1] 설정 변경
import os
import json
import numpy as np
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import torch
from datetime import datetime
from tqdm import tqdm
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 0. GPU 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {DEVICE}")

# 1. 설정 (Jina v3 적용)
MODEL_NAME = "jinaai/jina-embeddings-v3"

# === [로컬 환경 호환 경로] ===
DATA_DIR = Path("guardian_top100_scraping")
OUTPUT_DIR = Path("vector_bodyText")

# [중요] Jina v3는 모델이 크고 입력이 길어서 배치를 줄여야 합니다 (T4 GPU 기준)
BATCH_SIZE = 1
CHECKPOINT_FILE = OUTPUT_DIR / "checkpoint.json"

# 2. BodyText Chunking 설정 (Jina v3는 8192까지 가능)
# 8000으로 설정하면 뉴스 기사 99%는 안 잘리고 통째로 들어갑니다.
CHUNK_LENGTH = 8192
OVERLAP = 100

OUTPUT_DIR.mkdir(exist_ok=True)

# 3. Model과 Tokenizer 로드 (trust_remote_code=True 필수!)
logging.info("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = model.to(DEVICE)
model.eval()
logging.info("Model loaded successfully!")

Cell 2: 헬퍼 함수 정의 (핵심 로직)

In [ ]:
# [Cell 2] Helper Functions (Jina v3 Mean Pooling 적용)

def extract_person_name(filename):
    return filename.stem

def parse_pub_date(web_pub_date):
    try:
        dt = datetime.fromisoformat(web_pub_date.replace('Z', '+00:00'))
        return dt.strftime("%Y_%m_%d")
    except Exception:
        return None

def create_token_chunks(text, tokenizer, chunk_length=8000, overlap=100):
    if not text or text.strip() == '':
        return []

    # Jina tokenizer 사용 시 special token 처리
    tokens = tokenizer.encode(text, add_special_tokens=False)

    if not tokens:
        return []

    step = chunk_length - overlap
    token_chunks = []

    for i in range(0, len(tokens), step):
        chunk = tokens[i:i + chunk_length]
        token_chunks.append(chunk)
        if i + chunk_length >= len(tokens):
            break

    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks if chunk]
    return text_chunks

@torch.no_grad()
def generate_embeddings(texts, model, tokenizer, batch_size=4):
    """Batch 단위로 embedding 생성 (Jina v3: Mean Pooling + FP16)"""
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        # Jina v3는 8192까지 가능
        encoded = tokenizer(batch, padding=True, truncation=True,
                            max_length=8192, return_tensors='pt')
        encoded = {k: v.to(DEVICE) for k, v in encoded.items()}

        # FP16 적용
        with torch.amp.autocast('cuda'):
            outputs = model(**encoded)

            # === [변경] Mean Pooling ===
            # attention_mask를 고려하여 평균을 구합니다.
            input_mask_expanded = encoded['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
            sum_embeddings = torch.sum(outputs.last_hidden_state * input_mask_expanded, 1)
            sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            batch_embeddings = sum_embeddings / sum_mask

            # Normalize
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)

        embeddings.append(batch_embeddings.cpu().numpy().astype(np.float32))

    if not embeddings:
        return np.array([])

    return np.vstack(embeddings)

def mean_pool_embeddings(embeddings):
    if embeddings.size == 0:
        return None
    return np.mean(embeddings, axis=0)

Cell 3: 이어달리기(Checkpoint) 설정

In [ ]:
# Checkpoint 확인
processed_files = set()
if CHECKPOINT_FILE.exists():
    try:
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint = json.load(f)
            processed_files = set(checkpoint.get('processed_files', []))
            logging.info(f"Checkpoint found: {len(processed_files)} files already processed")
    except json.JSONDecodeError:
        logging.warning("Checkpoint file is corrupted. Starting fresh.")
        processed_files = set()

# 기존 저장된 데이터 로드
embeddings_file = OUTPUT_DIR / "embeddings.npy"
metadata_file = OUTPUT_DIR / "metadata.jsonl"

if embeddings_file.exists() and metadata_file.exists():
    logging.info("Loading existing data...")
    try:
        existing_embeddings = np.load(embeddings_file)
        existing_metadata = []
        with open(metadata_file, 'r', encoding='utf-8') as f:
            for line in f:
                existing_metadata.append(json.loads(line))

        if len(existing_embeddings) == len(existing_metadata):
            logging.info(f"Loaded {len(existing_metadata)} existing entries")
        else:
            logging.error("Data mismatch! embeddings and metadata have different lengths. Starting fresh.")
            existing_embeddings = None
            existing_metadata = []

    except Exception as e:
        logging.error(f"Error loading existing data: {e}. Starting fresh.")
        existing_embeddings = None
        existing_metadata = []
else:
    logging.info("Starting fresh (no existing data found)")
    existing_embeddings = None
    existing_metadata = []

# 모든 .jsonl 파일 수집 (처리되지 않은 파일만)
jsonl_files = sorted([f for f in DATA_DIR.glob("*.jsonl") if f.name not in processed_files])
logging.info(f"Total files to process: {len(jsonl_files)}")

Cell 4: 메인 루프 (데이터 처리 및 저장)

In [ ]:
# === [최종 Main Cell 4] 전체 데이터 실행 ===

logging.info("전체 데이터 임베딩 작업을 시작합니다.")

# 데이터 수집 및 embedding 생성 (인물별로 처리 및 즉시 저장)
for idx, file_path in enumerate(tqdm(jsonl_files, desc="Overall Progress")):
    person = extract_person_name(file_path)

    # 파일에서 기사 읽기
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            articles = [json.loads(line) for line in f]
    except Exception as e:
        logging.error(f"Failed to read {file_path}: {e}")
        continue

    # 현재 인물의 기사별 처리
    person_embeddings = []
    person_metadata = []

    for article in articles:
        body_text = article.get('bodyText', '')
        article_id = article.get('id')
        pub_date_raw = article.get('webPublicationDate')

        if not all([body_text, article_id, pub_date_raw]):
            continue # 필수 정보 누락

        # 1. 토큰 분할 (Chunking)
        text_chunks = create_token_chunks(body_text, tokenizer, CHUNK_LENGTH, OVERLAP)
        if not text_chunks:
            continue

        # 2. 임베딩 생성 (Batch 처리)
        chunk_embeddings = generate_embeddings(text_chunks, model, tokenizer, batch_size=BATCH_SIZE)

        # 3. 평균(Pooling)
        article_embedding = mean_pool_embeddings(chunk_embeddings)

        if article_embedding is not None:
            person_embeddings.append(article_embedding)
            person_metadata.append({
                'person': person,
                'article_id': article_id,
                'pub_date': parse_pub_date(pub_date_raw)
            })

    # --- 한 인물 처리가 끝나면 즉시 파일에 추가 (Append) ---
    if person_embeddings:
        person_embeddings_array = np.array(person_embeddings)

        # 기존 데이터와 합치기
        if existing_embeddings is not None:
            combined_embeddings = np.vstack([existing_embeddings, person_embeddings_array])
        else:
            combined_embeddings = person_embeddings_array

        combined_metadata = existing_metadata + person_metadata

        # 즉시 저장 (덮어쓰기)
        try:
            np.save(embeddings_file, combined_embeddings)
            with open(metadata_file, 'w', encoding='utf-8') as f:
                for meta in combined_metadata:
                    f.write(json.dumps(meta, ensure_ascii=False) + '\n')

            # 다음 반복을 위해 업데이트
            existing_embeddings = combined_embeddings
            existing_metadata = combined_metadata

        except Exception as e:
            logging.error(f"Failed to save data for {person}: {e}")
            continue

    # Checkpoint 업데이트 (저장 성공 시)
    processed_files.add(file_path.name)
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump({'processed_files': list(processed_files)}, f)

# --- 모든 작업 완료 ---
logging.info(f"\n All processing complete!")
if existing_embeddings is not None:
    logging.info(f"Final embeddings shape: {existing_embeddings.shape}")
    logging.info(f"Total articles: {len(existing_metadata)}")

# Checkpoint 삭제
if CHECKPOINT_FILE.exists():
    CHECKPOINT_FILE.unlink()
    logging.info("Checkpoint file removed.")